In [9]:
from skimage.io import imread, imsave
from skimage.filters import gaussian, sobel
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
import cv2
from skimage.color import yuv2rgb, rgb2yuv, gray2rgb, rgb2gray, rgb2lab, lab2rgb
from skimage.util import img_as_ubyte
import ipywidgets as widgets
import os 

INVESTIGATION_PATH = 'clahe'
os.makedirs(INVESTIGATION_PATH, exist_ok=True)
get_path = lambda file: os.path.join(INVESTIGATION_PATH, file)



image = data.chelsea()
image_yuv = rgb2yuv(image)

In [10]:
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
equalized = clahe.apply(gray)

In [11]:
imsave(get_path("gray.png"), gray)
imsave(get_path("equalized_gray.png"), equalized)

In [12]:
@widgets.interact_manual(cliplimit=widgets.FloatSlider(2, min=0, max=10, step=0.1), 
                         title_size=widgets.IntSlider(8, 2, 16),
                         save=False)
def investigate_clahe(cliplimit: float=2., title_size: int=8, save: bool=False):
    yuv_image = rgb2yuv(image)
    y_channel = (yuv_image[..., 0] * 255).astype(np.uint8)
    lab_image = rgb2lab(image)
    l_channel = (lab_image[..., 0] * 255).astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=cliplimit, tileGridSize=(title_size, title_size))
    equalized = clahe.apply(y_channel)
    equalized_l_channel = clahe.apply(l_channel)
    lab_image[..., 0] = equalized_l_channel / 255
    new_rgb_lab = lab2rgb(lab_image)
    yuv_image[..., 0] = equalized  / 255
    new_rgb = yuv2rgb(yuv_image)

    if save:
        imsave(get_path("new_equalized_rgb_lab.png"), new_rgb_lab)
        imsave(get_path("new_equalized_rgb.png"), new_rgb)

    difference = np.abs(new_rgb - image).mean()
    print("Difference: ", difference)
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 10))
    axes[0, 0].imshow(image)
    axes[0, 0].set_title("Initial image")
    axes[1, 0].hist(image.flatten())
    axes[2, 0].imshow(equalized_l_channel, cmap='gray')
    axes[2, 0].set_title("Equalized gray")

    axes[0, 1].imshow(new_rgb)
    axes[0, 1].set_title("Equalized image")
    axes[1, 1].hist((new_rgb * 255).astype(np.uint8).flatten())
    axes[2, 1].imshow(new_rgb_lab)
    axes[2, 1].set_title("Equalized gray -> RGB")



    plt.show()

interactive(children=(FloatSlider(value=2.0, description='cliplimit', max=10.0), IntSlider(value=8, descriptio…